# Imports

In [8]:
%reload_ext autoreload
%autoreload 2


import os
import sys
import torch
# from tqdm import tqdm
# import copy
# import h5py
import numpy as np
from IPython.display import display
# import pandas as pd
# from IPython.display import display
# import random
# import matplotlib.ticker as ticker
# import torchvision
# import matplotlib.pyplot as plt
# import sklearn
import pandas as pd
from stuned.utility.utils import (
    get_project_root_path,
    log_or_print
)
# from stuned.utility.logger import (
#     get_logger
# )


# local imports
sys.path.insert(
    0,
    os.path.dirname(os.path.abspath(''))
)
# from diverse_universe.cross_eval.eval import (
#     get_long_table
# )
from diverse_universe.local_datasets.imagenet_c import (
    IN_C_DATALOADERS_NAMES,
    # extract_in_c_paths
)
# from local_datasets.imagenet1k import (
#     IMAGENET2012_CLASSES_LIST,
#     DEFAULT_HUGGING_FACE_ACCESS_TOKEN_PATH,
#     get_imagenet_dataset,
#     collate_fn_hf
# )
# from local_datasets.transforms import (
#     make_transforms
# )
# from local_datasets.common import (
#     get_dataloaders,
#     make_dataloaders,
#     make_hdf5_dataloader_from_path,
#     make_cached_dataloaders
# )
# from local_datasets.utils import (
#     make_default_cache_path,
#     show_images_from_dataloader,
#     evaluate_model,
#     # make_default_test_transforms_imagenet,
#     evaluate_ensemble
# )
# from local_datasets.transforms import (
#     make_default_test_transforms_imagenet,
# )
# from utility.utils import (
#     apply_random_seed,
#     get_with_assert,
#     get_hash,
#     deterministically_subsample_indices_uniformly,
#     pretty_json,
#     append_dict,
#     read_json,
#     get_project_root_path,
#     extract_list_from_huge_string,
#     find_by_subkey
# )
# from local_datasets.from_h5 import (
#     make_hdf5_name
# )
# from local_datasets.wilds import (
#     metadata_to_group
# )
# from local_datasets.imagenet1k import (
#     IMAGENET_TRAIN_NUM_SAMPLES
# )
# from local_models.common import (
#     build_model,
#     make_ensemble_from_model_list,
#     wrap_model,
#     make_ensembles,
#     get_model,
#     make_ensemble_from_stacked_linear_layers
# )
# from local_models.utils import (
#     separate_classifier_and_featurizer
# )
# from local_datasets.model_vs_human import (
#     get_image_unnormalizer
# )
# from local_datasets.easy_robust import (
#     IN_C_DATALOADERS_NAMES,
#     extract_in_c_paths
# )
# from train_eval.models import (
#     is_ensemble,
#     make_uniform_soup
# )
# from train_eval.losses import (
#     METRICS,
#     div_ortega,
#     div_different_preds,
#     div_different_preds_per_sample,
#     div_continous_unique_per_sample,
#     div_continous_unique,
#     div_mean_logit,
#     div_max_logit,
#     div_mean_prob,
#     div_max_prob,
#     div_entropy,
#     div_different_preds_per_model,
#     focal_modifier,
#     div_var
# )
# from utility.for_notebooks import (
#     make_cross_dict,
#     make_ensembles_from_paths,
#     plot_table_from_cross_dict,
#     get_long_table,
#     format_number,
#     make_models_dict_from_huge_string,
#     evaluate_ensembles,
#     ensemble_from_multiple_paths
# )
# from local_models.common import (
#     get_model
# )
sys.path.pop(0)


# N_EPOCHS = 4
# N_SAMPLES = 96
# PATH_TO_CACHE = os.path.join(make_default_cache_path(), "ImageNet1k")
# HUGGING_FACE_ACCESS_TOKEN_PATH = os.path.expanduser("~/.config/hugging_face/hf.yaml")
# EXTENDED_METRICS = list(METRICS)
# EXTENDED_METRICS.extend([
#     ("div_ortega", div_ortega),
#     ("div_different_preds", div_different_preds)
# ])
# DEFAULT_ROUND = 9
# SCRATCH_LOCAL = os.path.join(
#     os.sep,
#     "scratch_local",
#     f"{os.environ.get('USER')}-{os.environ.get('SLURM_JOB_ID')}",
# )
DEFAULT_ROUND = 9
HYPERPARAM_PREFIX = "__hyperparam__"

# Functions

In [5]:
def plot_table_from_cross_dict(
    path_to_dict,
    value_name,
    round_to=None,
    to_show=True,
    multiplier=1,
    merge_map=None,
    logger=None,
    model_names=None
):
    def init_with_dicts(d, key):
        if key not in d:
            d[key] = {}

    def round_string(value, round_to=round_to, multiplier=multiplier, sep="+-"):
        mean, std = extract_mean_std(value, sep=sep)
        mean *= multiplier
        std *= multiplier
        return make_mean_std_str(mean, std, round_to, sep=sep)

    def as_sorted(iterable):
        return sorted(list(iterable), key=lambda x: x[0])

    def merge_dicts(dicts):
        res = {}
        for d in dicts:
            for key in d:
                if key not in res:
                    res[key] = d[key]
                else:
                    assert len(
                        set(d[key].keys()).intersection(set(res[key].keys()))
                    ) == 0
                    res[key] |= d[key]

        return res

    if isinstance(path_to_dict, list):
        cross_dict = merge_dicts([torch.load(path) for path in path_to_dict])
    else:
        cross_dict = torch.load(path_to_dict)

    res = {}
    hyperparams = {}
    # cross_dict: (i, j, k) model i -> dataset j -> stats k
    for model_name, model_dict in as_sorted(cross_dict.items()):

        init_with_dicts(res, model_name)
        init_with_dicts(hyperparams, model_name)

        if model_names is not None:
            assert isinstance(model_names, list)
            model_names.append(model_name)

        for dataset_name, stats in as_sorted(model_dict.items()):

            if HYPERPARAM_PREFIX in dataset_name:
                split = dataset_name.split(HYPERPARAM_PREFIX)
                assert len(split) == 2
                hyperparams[model_name][split[1]] = stats
                continue

            assert isinstance(stats, dict)
            if value_name not in stats:
                log_or_print(
                    f"For {model_name} on {dataset_name} value {value_name} "
                    f"not found among {stats.keys()}",
                    logger,
                    auto_newline=True
                )
            value = stats.get(value_name)

            if value is not None and round_to is not None:
                value = round_string(value)

            res[model_name][dataset_name] = value

        if merge_map is not None:
            # merge_map: merged_name -> [<names of datasets to merge>]
            for merged_dataset in merge_map.keys():
                datasets_to_merge_names = merge_map[merged_dataset]
                datasets_to_merge_values = []
                for dataset_to_merge in datasets_to_merge_names:
                    datasets_to_merge_values.append(
                        res[model_name].pop(dataset_to_merge)
                    )

                res[model_name][merged_dataset] = merge_mean_std(
                   datasets_to_merge_values,
                   round_to
                )

    df = pd.DataFrame.from_dict(res, orient='index')
    hyperparams = pd.DataFrame.from_dict(hyperparams, orient='index')

    if to_show:
        log_or_print(
            f"Table for {value_name}",
            logger,
            auto_newline=True
        )
        display(df)
    return df, hyperparams


def extract_mean_std(value, sep="+-"):
    if value is None:
        return None, None
    mean, std = map(float, value.split(sep))
    return mean, std

def make_mean_std_str(mean, std, round_to, sep="+-"):
    if mean is None or std is None:
        assert std is None and mean is None
        return None

    return f"{round(mean, round_to)} {sep} {round(std, round_to)}"


def merge_mean_std(values, round_to, sep="+-"):

    means, stds = [], []
    for value in values:
        mean, std = extract_mean_std(value, sep=sep)
        if mean is not None:
            means.append(mean)
        if std is not None:
            stds.append(std)

    res_mean, res_std = None, None
    if len(means) > 0:
        res_mean = np.array(means).mean()
    if len(stds) > 0:
        res_std = np.sqrt((np.array(stds) ** 2).sum())
    return make_mean_std_str(res_mean, res_std, round_to, sep=sep)


def get_long_table(
    pths,
    metric_names,
    axis,
    row_names,
    col_names,
    format_func,
    group_by_columns=False,
    merge_map=None
):

    dfs = []
    hyperparams = None

    for metric_name in metric_names:

        if row_names is None:
            extracted_model_names = []
        else:
            extracted_model_names = None

        if isinstance(metric_name, tuple):
            assert len(metric_name) == 2
            multiplier = metric_name[1]
            metric_name = metric_name[0]
        else:
            multiplier = 1

        df, hyperparams_df = plot_table_from_cross_dict(
            pths,
            metric_name,
            round_to=DEFAULT_ROUND,
            to_show=False,
            multiplier=multiplier,
            merge_map=merge_map,
            model_names=extracted_model_names
        )
        if hyperparams is None:
            hyperparams = hyperparams_df
        else:
            assert hyperparams.equals(hyperparams_df)

        dfs.append(df)

        if extracted_model_names is not None:
            row_names = extracted_model_names

    return merge_dfs(
        dfs,
        axis,
        row_names=row_names,
        col_names=col_names,
        format_func=format_func,
        group_by_columns=group_by_columns,
        metric_names=metric_names,
        just_concat=[hyperparams_df]
    )


def merge_dfs(
    dfs,
    axis,
    row_names=None,
    col_names=None,
    format_func=None,
    group_by_columns=False,
    insert_empty_cols=True,
    metric_names=None,
    just_concat=[]
):

    def insert_row_above(df, row_as_list):

        df = pd.concat(
            [pd.DataFrame([row_as_list], columns=df.columns), df]
        )
        return df

    if metric_names is not None:
        assert len(metric_names) == len(dfs)

    if group_by_columns:
        pre_groupped = {}
    else:
        pre_groupped = None
    for i in range(len(dfs)):
        if row_names is not None:
            dfs[i] = dfs[i].loc[row_names]
        if col_names is not None:
            dfs[i] = dfs[i].loc[:, col_names]
        if format_func is not None:
            dfs[i] = dfs[i].applymap(format_func)

        if metric_names is not None:
            row_as_list = [metric_names[i]] * len(dfs[i].columns)
            dfs[i] = insert_row_above(
                dfs[i],
                row_as_list
            )

        if pre_groupped is not None:
            for column_name in dfs[i].columns:
                if column_name not in pre_groupped:
                    pre_groupped[column_name] = []

                pre_groupped[column_name].append(
                    dfs[i][[column_name]]
                )

    if pre_groupped is None:
        dfs_to_concat = dfs
    else:
        dfs_to_concat = [
                pd.concat(
                    [
                        single_column
                            for single_column
                                in pre_groupped_for_column
                    ],
                    axis=axis
                )
                        for pre_groupped_for_column
                        in pre_groupped.values()
            ]

    # to have empty row on top and be well aligned with other dfs
    for i in range(len(just_concat)):
        just_concat[i] = insert_row_above(
            just_concat[i],
            ["hyperparam"] * len(just_concat[i].columns)
        )

    dfs_to_concat = just_concat + dfs_to_concat
    if insert_empty_cols:
        dfs_with_inserted = []
        for i, df in enumerate(dfs_to_concat):
            dfs_with_inserted.append(df)
            if i + 1 != len(dfs_to_concat):
                dfs_with_inserted.append(pd.Series(dtype='int'))
        dfs_to_concat = dfs_with_inserted

    res_df = pd.concat(dfs_to_concat, axis=axis)
    return res_df


def format_number(pm_str):
    if pm_str is None:
        return pm_str
    number = pm_str.split('+-')[0]
    return abs(float(number))

# Visualized evaluation results

In [6]:
_path = os.path.join(".", "/mnt/qb/work/oh/arubinstein17/cache/results/NIPS2024/deit3b_ood_gen1605.pkl")

all_corruption_names = list(IN_C_DATALOADERS_NAMES.values())

df = get_long_table(
    [
        _path,
    ],
    [
        # ("submodel_0", 100),
        # ("submodel_1", 100),
        # ("submodel_2", 100),
        # ("submodel_3", 100),
        # ("submodel_4", 100),
        ("best_single_model", 100),
        # ("mean_single_model", 100),
        ("ensemble", 100),
        ("soup", 100),
        "div_different_preds",
        "div_continous_unique",
        "var"
    ],
    axis=1,
    # row_names=list(budget_models_dict.keys()),
    row_names=None,
    # row_names=["ce_parallel"],
    col_names=[
        "in_val",
        # "in_val_45k_easy",
        # "in_val_5k_hard",
        "imagenet_a",
        "imagenet_r",
        # "in_train_3068_hard",
        "C-1",
        "C-5",
        "iNaturalist",
        "OpenImages"
        # "stylized"
        # "fog_1",
        # "fog_5",
        # "defocus_blur_1",
        # "defocus_blur_5"
    ],
    format_func=format_number,
    merge_map={
        "C-1": [corruption + "_1" for corruption in all_corruption_names],
        "C-5": [corruption + "_5" for corruption in all_corruption_names],
        "IN-9": ["mixed_rand", "mixed_next"]
    }
)

For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on OpenImages value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on brightness_1 value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on brightness_5 value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on contrast_1 value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on contrast_5 value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lambda_2.00E-04 on defocus_blur_1 value best_single_model not found among dict_keys(['ensemble', 'submodel_0', 'divs', 'cont_unique'])
For deit3b_a2d_inr_50_manual_lam

# Display df

In [9]:
display(df)

,manual_lambda,reg_weight,batch_size,n_models,lambda,0,in_val,imagenet_a,imagenet_r,C-1,...,iNaturalist,OpenImages,5,in_val,imagenet_a,imagenet_r,C-1,C-5,iNaturalist,OpenImages
0,hyperparam,hyperparam,hyperparam,hyperparam,hyperparam,NaN,"(best_single_model, 100)","(best_single_model, 100)","(best_single_model, 100)","(best_single_model, 100)",...,div_continous_unique,div_continous_unique,NaN,var,var,var,var,var,var,var
2105_deit3b_a2d_inr_50_manual_lambda_2.00E-04,2.00E-04,NaN,NaN,NaN,NaN,NaN,82.842,38.893333,46.116667,74.8648,...,1.944735,1.86334,NaN,0.000001,0.0,0.0,0.000001,0.0,0.0,0.0
2105_deit3b_div_inr_50_manual_lambda_2.00E-05_reg_weight_1.00E-02,2.00E-05,1.00E-02,NaN,NaN,NaN,NaN,82.868,38.773333,46.203333,74.885067,...,1.941597,1.859734,NaN,0.0,0.0,0.0,0.000001,0.0,0.0,0.0
2205_deit3b_budget_batches_n_models_5_manual_lambda_0.5_batch_size_128,0.5,NaN,128,5,NaN,NaN,82.572,25.813333,36.66,70.378533,...,4.106618,3.528619,NaN,0.000036,0.0,0.0,0.000101,0.000385,0.000598,0.000428
2205_deit3b_budget_batches_n_models_5_manual_lambda_0.5_batch_size_16,0.5,NaN,16,5,NaN,NaN,84.502,35.066667,43.17,75.2188,...,3.492439,2.97349,NaN,0.000032,0.0,0.0,0.000083,0.000294,0.00039,0.000277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
deit3b_shallow_ens_v2_50_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.973333,45.1,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
deit3b_shallow_ens_v2_single_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.88,44.713333,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
deit3b_vary_hp_v2_,NaN,NaN,NaN,NaN,NaN,NaN,85.414,38.48,45.423333,77.432667,...,1.412057,1.376913,NaN,0.000005,0.0,0.0,0.000009,0.000013,0.000012,0.000012
deit3b_vary_hp_v2_50_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.36,45.39,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


# Save to csv

In [10]:
tmp_folder = os.path.join(get_project_root_path(), "tmp")
os.makedirs(tmp_folder, exist_ok=True)
df.to_csv(os.path.join(tmp_folder, "tmp_csv.csv"))